## 1. Introduction
<p><img src="https://assets.datacamp.com/production/project_1197/img/google_play_store.png" alt="Google Play logo"></p>
<p>Mobile apps are everywhere. They are easy to create and can be very lucrative from the business standpoint. Specifically, Android is expanding as an operating system and has captured more than 74% of the total market<sup><a href="https://www.statista.com/statistics/272698/global-market-share-held-by-mobile-operating-systems-since-2009">[1]</a></sup>. </p>
<p>The Google Play Store apps data has enormous potential to facilitate data-driven decisions and insights for businesses. In this notebook, we will analyze the Android app market by comparing ~10k apps in Google Play across different categories. We will also use the user reviews to draw a qualitative comparision between the apps.</p>
<p>The dataset you will use here was scraped from Google Play Store in September 2018 and was published on <a href="https://www.kaggle.com/lava18/google-play-store-apps">Kaggle</a>. Here are the details: <br>
<br></p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/apps.csv</b></div>
This file contains all the details of the apps on Google Play. There are 9 features that describe a given app.
<ul>
    <li><b>App:</b> Name of the app</li>
    <li><b>Category:</b> Category of the app. Some examples are: ART_AND_DESIGN, FINANCE, COMICS, BEAUTY etc.</li>
    <li><b>Rating:</b> The current average rating (out of 5) of the app on Google Play</li>
    <li><b>Reviews:</b> Number of user reviews given on the app</li>
    <li><b>Size:</b> Size of the app in MB (megabytes)</li>
    <li><b>Installs:</b> Number of times the app was downloaded from Google Play</li>
    <li><b>Type:</b> Whether the app is paid or free</li>
    <li><b>Price:</b> Price of the app in US$</li>
    <li><b>Last Updated:</b> Date on which the app was last updated on Google Play </li>

</ul>
</div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:20px"><b>datasets/user_reviews.csv</b></div>
This file contains a random sample of 100 <i>[most helpful first](https://www.androidpolice.com/2019/01/21/google-play-stores-redesigned-ratings-and-reviews-section-lets-you-easily-filter-by-star-rating/)</i> user reviews for each app. The text in each review has been pre-processed and passed through a sentiment analyzer.
<ul>
    <li><b>App:</b> Name of the app on which the user review was provided. Matches the `App` column of the `apps.csv` file</li>
    <li><b>Review:</b> The pre-processed user review text</li>
    <li><b>Sentiment Category:</b> Sentiment category of the user review - Positive, Negative or Neutral</li>
    <li><b>Sentiment Score:</b> Sentiment score of the user review. It lies between [-1,1]. A higher score denotes a more positive sentiment.</li>

</ul>
</div>
<p>From here on, it will be your task to explore and manipulate the data until you are able to answer the three questions described in the instructions panel.<br></p>

In [128]:
import pandas as pd
import numpy as np

In [129]:
apps=pd.read_csv('datasets/apps.csv')

In [130]:
apps.drop_duplicates(subset='App',keep=False,inplace=True)

In [131]:
char_to_remove = [',','+']
for item in char_to_remove:
    apps['Installs'] = apps['Installs'].apply(lambda x: x.replace(item,''))

In [132]:
apps['Installs']=apps.Installs.astype(int)

In [133]:
cat_count=apps['Category'].value_counts()

In [134]:
app_category_info=apps.groupby('Category')[['Rating','Price']].mean()

In [135]:
app_category_info['Number of apps']=cat_count[cat_count.index.sort_values()]

In [136]:
app_category_info.reset_index(inplace=True)

In [137]:
app_category_info.columns=['Category','Average rating','Average price','Number of apps']

In [138]:
columns=['Category','Number of apps','Average price','Average rating']
app_category_info=app_category_info[columns]
app_category_info

,Category,Number of apps,Average price,Average rating
0,ART_AND_DESIGN,64,0.093281,4.357377
1,AUTO_AND_VEHICLES,85,0.158471,4.190411
2,BEAUTY,53,0.000000,4.278571
3,BOOKS_AND_REFERENCE,222,0.539505,4.344970
4,BUSINESS,420,0.417357,4.098479
5,COMICS,56,0.000000,4.181481
6,COMMUNICATION,315,0.263937,4.121484
7,DATING,171,0.160468,3.970149
8,EDUCATION,119,0.150924,4.364407
9,ENTERTAINMENT,102,0.078235,4.135294


In [139]:
reviews=pd.read_csv('datasets/user_reviews.csv')

In [140]:
reviews.head()

,App,Review,Sentiment Category,Sentiment Score
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25
2,10 Best Foods for You,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40
4,10 Best Foods for You,Best idea us,Positive,1.00


In [141]:
apps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,"January 7, 2018"
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,"January 15, 2018"
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,"August 1, 2018"
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,"June 8, 2018"
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,"June 20, 2018"


In [142]:
reviews_cat=reviews.groupby('App')[['Sentiment Score']].mean()

In [143]:
reviews_cat

,Sentiment Score
App,
10 Best Foods for You,0.470733
104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,0.392405
11st,0.181294
1800 Contacts - Lens Store,0.318145
1LINE – One Line with One Touch,0.196290
2018Emoji Keyboard 😂 Emoticons Lite -sticker&gif,0.449566
21-Day Meditation Experience,0.258014
"2Date Dating App, Love and matching",0.280267
2GIS: directory & navigator,0.223129


In [144]:
free_finance_apps=apps[(apps['Category'] == 'FINANCE') & (apps['Type'] == 'Free')]

In [145]:
free_finance_apps_sentiments= free_finance_apps.merge(reviews_cat , on='App')

In [151]:
top_10_user_feedback=free_finance_apps_sentiments.sort_values(by=['Sentiment Score','Installs','Rating'],ascending=False)[:10]['']

In [152]:
top_10_user_feedback

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Last Updated,Sentiment Score
4,BBVA Spain,FINANCE,4.2,36746,NaN,5000000,Free,0.0,"July 24, 2018",0.515086
36,Associated Credit Union Mobile,FINANCE,4.7,3290,12.0,50000,Free,0.0,"May 30, 2018",0.388093
34,BankMobile Vibe App,FINANCE,4.3,14627,23.0,1000000,Free,0.0,"July 9, 2018",0.353455
44,A+ Mobile,FINANCE,3.9,730,6.3,10000,Free,0.0,"June 26, 2018",0.329592
29,Current debit card and app made for teens,FINANCE,4.3,685,21.0,50000,Free,0.0,"August 3, 2018",0.327258
8,BZWBK24 mobile,FINANCE,4.5,64959,39.0,1000000,Free,0.0,"June 18, 2018",0.326883
12,"Even - organize your money, get paid early",FINANCE,4.8,12304,21.0,100000,Free,0.0,"August 2, 2018",0.283929
7,Credit Karma,FINANCE,4.7,706301,NaN,10000000,Free,0.0,"July 31, 2018",0.270052
45,Fortune City - A Finance App,FINANCE,4.6,49275,91.0,500000,Free,0.0,"July 17, 2018",0.266966
19,Branch,FINANCE,4.6,69973,3.8,1000000,Free,0.0,"July 23, 2018",0.264230
